# Chisel Project

Here we get familar with basic chisel concepts and progressively build a very simple CPU in chisel and run assembly code on it using the scala compiler we developed in the previous tutorial.

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/Users/navaneeth/scratch/chisel-miniproject/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.experimental._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.experimental._

import chisel3.tester._

import chisel3.tester.RawTester.test

In [50]:
case class RegisterFile(readPorts: Int) extends Module {
    require(readPorts >= 0)
    val io = IO(new Bundle {
        val wen   = Input(Bool())
        val waddr = Input(UInt(5.W))
        val wdata = Input(UInt(32.W))
        val raddr = Input(Vec(readPorts, UInt(5.W)))
        val rdata = Output(Vec(readPorts, UInt(32.W)))
    })
    
    // A Register of a vector of UInts
    val reg = RegInit(VecInit(Seq.fill(32)(0.U(32.W))))
    
    when (io.wen) {
        reg(io.waddr) := io.wdata
    }
    
    for (i <- 0 until readPorts) {
        when (io.raddr(i) === 0.U) {
            io.rdata(i) := 0.U
        } .otherwise {
            io.rdata(i) := reg(io.raddr(i))
        }
    }
}

defined class RegisterFile

In [4]:
test(new RegisterFile(2) ) { c =>
  def readExpect(addr: Int, value: Int, port: Int = 0): Unit = {
    c.io.raddr(port).poke(addr.U)
    c.io.rdata(port).expect(value.U)
  }
  def write(addr: Int, value: Int): Unit = {
    c.io.wen.poke(true.B)
    c.io.wdata.poke(value.U)
    c.io.waddr.poke(addr.U)
    c.clock.step(1)
    c.io.wen.poke(false.B)
  }
  // everything should be 0 on init
  for (i <- 0 until 32) {
    readExpect(i, 0, port = 0)
    readExpect(i, 0, port = 1)
  }

  // write 5 * addr + 3
  for (i <- 0 until 32) {
    write(i, 5 * i + 3)
  }

  // check that the writes worked
  for (i <- 0 until 32) {
    readExpect(i, if (i == 0) 0 else 5 * i + 3, port = i % 2)
  }
}

Elaborating design...
Done elaborating.
test RegisterFile Success: 0 tests passed in 34 cycles in 0.115678 seconds 293.92 Hz


In [5]:
case class RVCompiler() {
    val supported_ops = Array("addi", "add", "sub", "mul")
    val supported_regs = Array("x0", "x1", "x2", "x3")
    val reg_map = Map("x0" -> 0, "x1" -> 1, "x2" -> 2, "x3" -> 3)

    def compile(asm_string: String): Int = {
        val opcode = asm_string.split(" ")(0).toLowerCase().stripSuffix(",")
        require(supported_ops.contains(opcode))
        val operands = List((asm_string.split(" ")(1).stripSuffix(",")), (asm_string.split(" ")(2).stripSuffix(",")), (asm_string.split(" ")(3)))
        require(operands.size == 3)
        var inst: Int = 0

        opcode match {
            case "addi" => {
                check_i_type(operands)
                inst = ITypeInstr(operands(1), operands(0), operands(2))
            }
            case "add"  => {
                check_r_type(operands)
                inst = RTypeInstr(operands(1), operands(2), operands(0), 0x00)
            }
            case "sub"  => { 
                check_r_type(operands)
                inst = RTypeInstr(operands(1), operands(2), operands(0), 0x20)
            }
            case "mul"  => {
                check_r_type(operands)
                inst = RTypeInstr(operands(1), operands(2), operands(0), 0x01)
            }
            case _ => throw new Exception("Unsupported instruction") 
        }
        return inst
    }
    
    def RTypeInstr(rs1: String, rs2: String, rd: String, funct7: Int): Int = {
        val bin_val: Int = funct7 << 25 | reg_map(rs2) << 20 | reg_map(rs1) << 15 | 0 << 12 | reg_map(rd) << 7 | 0x33
        return bin_val
    }
    
    def ITypeInstr(rs1: String, rd: String, imm: String): Int = {
        val imm_val = imm.toInt
        val bin_val: Int = imm_val << 20 | reg_map(rs1) << 15 | 0 << 12 | reg_map(rd) << 7 | 0x13
        return bin_val
    }
    
    def check_r_type(operands: List[String]) = {
        require(supported_regs.contains(operands(0)))
        require(supported_regs.contains(operands(1)))
        require(supported_regs.contains(operands(2)))
    }
    
    def check_i_type(operands: List[String]) = {
        val imm_val = operands(2).toInt
        require(imm_val >= -2048 && imm_val <= 2047)
        require(supported_regs.contains(operands(0)))
        require(supported_regs.contains(operands(1)))
    }
}


defined class RVCompiler

In [81]:
class CrayRV32() extends Module {
    val io = IO(new Bundle {
        val i_bus = Input(UInt(32.W))
        val illegal_instr = Output(Bool())
        val raddr = Input(UInt(2.W))
        val rdata = Output(UInt(32.W))
    })
    
    val reg_file = Module(RegisterFile(3))
    io.rdata := reg_file.io.rdata(0)
    reg_file.io.raddr(0) := io.raddr
    

    val opcode = io.i_bus(6, 0)
    val rd     = io.i_bus(11, 7)
    val funct3 = io.i_bus(14, 12)
    val rs1    = io.i_bus(19, 15)
    val rs2    = io.i_bus(24, 20)
    val funct7 = io.i_bus(31, 25)
    val imm    = io.i_bus(31, 20)
    
//     printf("instr: 0x%x, opcode = 0x%x\n",io.i_bus, opcode)
    
    reg_file.io.waddr := rd
    
    when(opcode === 0x33.U) {  //How to put this in a clocked always@??
//         printf(p"R Type rd:$rd rs1:$rs1 rs2:$rs2\n")
        reg_file.io.raddr(1) := rs1
        reg_file.io.raddr(2) := rs2
        reg_file.io.wen      := false.B
        io.illegal_instr     := true.B
        reg_file.io.wdata    := 0.U
        switch(funct7) {  //How to put default case??
            is(0x00.U) { //ADD
                printf(p"ADD  [$rd] = [$rs1] + [$rs2]\n") //Print during simulation Scala style formatting
                reg_file.io.wen      := true.B
                reg_file.io.wdata    := reg_file.io.rdata(1) + reg_file.io.rdata(2)
                io.illegal_instr := false.B
            }
            is(0x01.U) { //MUL
                printf("MUL  [%d] = [%d] * [%d]\n", rd, rs1, rs2) //Print during simulation C style formatting
                reg_file.io.wen      := true.B
                reg_file.io.wdata    := reg_file.io.rdata(1) * reg_file.io.rdata(2)
                io.illegal_instr := false.B
            }
            is(0x20.U) { //SUB
                printf("SUB  [%d] = [%d] - [%d]\n", rd, rs1, rs2)
                reg_file.io.wen      := true.B
                reg_file.io.wdata    := reg_file.io.rdata(1) - reg_file.io.rdata(2)
                io.illegal_instr := false.B
            }
        }
    }.elsewhen(opcode === 0x13.U) { //ADDI
        io.illegal_instr := false.B
        printf("ADDI [%d] = [%d] + %d\n", rd, rs1, imm)
        reg_file.io.raddr(1) := rs1
        reg_file.io.raddr(2) := 0.U
        reg_file.io.wen      := true.B
        reg_file.io.wdata    := imm + reg_file.io.rdata(1)
    }.otherwise {
        io.illegal_instr := true.B
        reg_file.io.wen  := false.B
        reg_file.io.raddr(1) := 0.U
        reg_file.io.raddr(2) := 0.U
        reg_file.io.wdata    := 0.U
    }   
}

//Print the verilog
// println(getVerilog(new CrayRV32()))

defined class CrayRV32

In [82]:
//Simple sanity check
test(new CrayRV32()) { c =>
    val compiler = RVCompiler()
    
    val instr_list = Seq(
        "addi x1, x0, 30",
        "addi x2, x0, 10",
        "addi x3, x0, 10",
        "add x3, x1, x2",
        "sub x3, x1, x2",
        "mul x3, x1, x2"
    )
    
    val reg_golden = Seq(
        Seq(1, 30),
        Seq(2, 10),
        Seq(3, 10),
        Seq(3, 40),
        Seq(3, 20),
        Seq(3, 300)
    )
    
    def readExpect(addr: Int, value: Int): Unit = {
        c.io.raddr.poke(addr.U)
//         println(s"${c.io.rdata.peek().litValue.toInt} ")
        c.io.rdata.expect(value.U)
    }
    
    def check_golden(indx: Int): Unit = {
        readExpect(reg_golden(indx)(0), reg_golden(indx)(1))
    }
        
    instr_list.zipWithIndex.foreach{
        case (instr, idx) => {
            val inst_enc = compiler.compile(instr)
            c.io.i_bus.poke(inst_enc.U)
            c.io.illegal_instr.expect(false.B)
            c.clock.step(1)
            check_golden(idx)
        }
    }
    
//     for (instr <- instr_list) {
//         val inst_enc = compiler.compile(instr)
//         c.io.i_bus.poke(inst_enc.U)
//         c.clock.step(1)
//         print_regfile()
//     }
}



Elaborating design...
Done elaborating.
ADDI [  1] = [  0] +    30
ADDI [  2] = [  0] +    10
ADDI [  3] = [  0] +    10
ADD  [  3] = [  1] + [  2]
SUB  [  3] = [  1] - [  2]
MUL  [  3] = [  1] * [  2]
test CrayRV32 Success: 0 tests passed in 8 cycles in 0.025158 seconds 317.98 Hz
